In [1]:
# from roboflow import Roboflow
# rf = Roboflow(api_key="T7uEFXRKxnlKwmjBGbBD")
# project = rf.workspace("ongsa-23tub").project("noocr")
# version = project.version(3)
# dataset = version.download("yolov8")
                

In [2]:
# !nvidia-smi

In [3]:
# def initialize_csv():
#     # Create or update CSV file with initial counts
#     csv_path = 'object_counts.csv'
#     if not os.path.exists(csv_path):
#         with open(csv_path, 'w', newline='') as file:
#             writer = csv.writer(file)
#             writer.writerow(['Object', 'ID', 'NUM'])
#             writer.writerow(['SOBO_WP_4200', 'WP0010', '0'])
#             writer.writerow(['YAMANO_LPP_20', 'AP0020', '0'])
#     return csv_path
# initialize_csv()

In [4]:
import os
import csv
from  datetime import datetime
from ultralytics import YOLO
import cv2
import numpy as np

def train_model():
    # Initialize YOLO model
    model= YOLO('yolov8n.pt')
    
    # Train the model
    results = model.train(
        data=r'C:\Users\gifly\Desktop\DataScience\Project\NoOCR-3\data.yaml',
        epochs=50,
        batch=8,
        imgsz=512,
        device=0,
        patience=5,
    )
    
    return model

def initialize_csv():
    # Create or update CSV file with initial counts
    csv_path = 'object_counts.csv'
    if not os.path.exists(csv_path):
        with open(csv_path, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['Object', 'ID', 'NUM'])
            writer.writerow(['SOBO_WP_4200', 'WP0010', '0'])
            writer.writerow(['YAMANO_LPP_20', 'AP0020', '0'])
    return csv_path

def update_csv_counts(new_counts):
    csv_path = 'object_counts.csv'
    
    # Check if the CSV file exists
    if os.path.exists(csv_path):
        # Read the existing CSV file
        with open(csv_path, 'r') as file:
            reader = csv.DictReader(file)
            rows = list(reader)
        
        # Update the counts
        for row in rows:
            if row['Object'] == 'SOBO_WP-4200':
                row['NUM'] = str(int(row['NUM']) + new_counts['SOBO_WP-4200'])
            elif row['Object'] == 'YAMANO_LPP-20':
                row['NUM'] = str(int(row['NUM']) + new_counts['YAMANO_LPP-20'])
        
        # Write the updated counts back to the CSV file
        with open(csv_path, 'w', newline='') as file:
            writer = csv.DictWriter(file, fieldnames=['Object', 'ID', 'NUM'])
            writer.writeheader()
            writer.writerows(rows)
    else:
        # Create a new CSV file if it doesn't exist
        with open(csv_path, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['Object', 'ID', 'NUM'])
            writer.writerow(['SOBO_WP-4200', 'WP0010', str(new_counts['SOBO_WP-4200'])])
            writer.writerow(['YAMANO_LPP-20', 'AP0020', str(new_counts['YAMANO_LPP-20'])])

def detect_and_count(model, image_path):
    # Read image
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Could not read image {image_path}")
        return None
    
    # Perform detection
    results = model(image)
    
    # Initialize counts
    counts = {'SOBO_WP-4200': 0, 'YAMANO_LPP-20': 0}
    
    # Process detections
    for result in results:
        boxes = result.boxes
        for box in boxes:
            # Get class name
            class_id = int(box.cls[0])
            class_name = model.names[class_id]
            
            # Increment count
            counts[class_name] += 1
            
            # Draw bounding box and label
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = float(box.conf[0])
            
            # Draw rectangle
            cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
            
            # Add label
            label = f"{class_name} {conf:.2f}"
            cv2.putText(image, label, (x1, y1 - 10), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    # Add total counts to image
    count_text = f"SOBO_WP-4200: {counts['SOBO_WP-4200']}, YAMANO_LPP-20: {counts['YAMANO_LPP-20']}"
    cv2.putText(image, count_text, (10, 30), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    
    # Save annotated image
    output_path = f"detected_{os.path.basename(image_path)}"
    cv2.imwrite(output_path, image)
    
    return counts

def main():
    # Step 1: Train the model
    print("Training model...")
    model = train_model()
    
    # Step 2: Initialize CSV
    csv_path = initialize_csv()
    print(f"CSV file initialized at {csv_path}")
    
    # Step 3: Process test images
    test_dir = "test/images"
    if os.path.exists(test_dir):
        for image_file in os.listdir(test_dir):
            if image_file.lower().endswith(('.png', '.jpg', '.jpeg')):
                image_path = os.path.join(test_dir, image_file)
                print(f"\nProcessing {image_path}")
                
                # Detect and count objects
                counts = detect_and_count(model, image_path)
                if counts:
                    # Update CSV with new counts
                    update_csv_counts(counts)
                    print(f"Updated counts: SOBO_WP-4200={counts['SOBO_WP-4200']}, YAMANO_LPP-20={counts['YAMANO_LPP-20']}")
    
    print("\nProcessing complete!")


if __name__ == "__main__":
    main()


Training model...
New https://pypi.org/project/ultralytics/8.3.111 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.108  Python-3.11.2 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce GTX 1660 Ti, 6144MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=C:\Users\gifly\Desktop\DataScience\Project\NoOCR-3\data.yaml, epochs=50, time=None, patience=5, batch=8, imgsz=512, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks

train: Scanning C:\Users\gifly\Desktop\DataScience\Project\NoOCR-3\train\labels.cache... 1500 images, 164 backgrounds, 0 corrupt: 100%|██████████| 1500/1500 [00:00<?, ?it/s]
val: Scanning C:\Users\gifly\Desktop\DataScience\Project\NoOCR-3\valid\labels.cache... 221 images, 15 backgrounds, 0 corrupt: 100%|██████████| 221/221 [00:00<?, ?it/s]


Plotting labels to runs\detect\train5\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 512 train, 512 val
Using 8 dataloader workers
Logging results to runs\detect\train5
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/50      2.28G     0.9957      1.589       1.18         11        512: 100%|██████████| 188/188 [00:23<00:00,  7.90it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 14/14 [00:05<00:00,  2.70it/s]

                   all        221        422      0.898      0.902      0.966      0.637


...

      21/50      2.42G     0.7651     0.4597       1.04          9        512: 100%|██████████| 188/188 [00:25<00:00,  7.34it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 14/14 [00:01<00:00,  7.70it/s]

                   all        221        422       0.97      0.998      0.984      0.814
EarlyStopping: Training stopped early as no improvement observed in last 5 epochs. Best results observed at epoch 16, best model saved as best.pt.
To update EarlyStopping(patience=5) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



21 epochs completed in 0.158 hours.
Optimizer stripped from runs\detect\train5\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train5\weights\best.pt, 6.2MB

Validating runs\detect\train5\weights\best.pt...
Ultralytics 8.3.108  Python-3.11.2 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce GTX 1660 Ti, 6144MiB)
Model summary (fused): 72 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 14/14 [00:02<00:00,  6.01it/s]


                   all        221        422      0.975      0.995      0.989      0.816
          SOBO_WP_4200        132        204      0.975          1      0.991       0.79
         YAMANO_LPP_20        146        218      0.975      0.991      0.987      0.841
Speed: 0.2ms preprocess, 2.8ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to runs\detect\train5
CSV file initialized at object_counts.csv

Processing complete!


In [5]:
# import os
# import csv
# from datetime import datetime
# from ultralytics import YOLO
# import cv2
# import numpy as np

# def train_model():
#     # Initialize YOLO model
#     model = YOLO('yolov8n.pt')
    
#     # Train the model
#     results = model.train(
#         data=r'C:\Users\gifly\Desktop\DataScience\Project\NoOCR\data.yaml',
#         epochs=5,
#         batch=16,
#         imgsz=640,
#         device=0
#     )
    
#     return model

# def initialize_csv():
#     # Create or update CSV file with initial counts
#     csv_path = 'object_counts.csv'
#     if not os.path.exists(csv_path):
#         with open(csv_path, 'w', newline='') as file:
#             writer = csv.writer(file)
#             writer.writerow(['Object', 'ID', 'NUM'])
#             writer.writerow(['Dryer', 'D0001', '0'])
#             writer.writerow(['IRON', 'I0002', '0'])
#     return csv_path

# def update_csv_counts(counts):
#     csv_path = 'object_counts.csv'
#     with open(csv_path, 'w', newline='') as file:
#         writer = csv.writer(file)
#         writer.writerow(['Object', 'ID', 'NUM'])
#         writer.writerow(['Dryer', 'D0001', str(counts['Dryer'])])
#         writer.writerow(['IRON', 'I0002', str(counts['IRON'])])

# def detect_and_count(model, image_path):
#     # Read image
#     image = cv2.imread(image_path)
#     if image is None:
#         print(f"Error: Could not read image {image_path}")
#         return None
    
#     # Perform detection
#     results = model(image)
    
#     # Initialize counts
#     counts = {'Dryer': 0, 'IRON': 0}
    
#     # Process detections
#     for result in results:
#         boxes = result.boxes
#         for box in boxes:
#             # Get class name
#             class_id = int(box.cls[0])
#             class_name = model.names[class_id]
            
#             # Increment count
#             counts[class_name] += 1
            
#             # Draw bounding box and label
#             x1, y1, x2, y2 = map(int, box.xyxy[0])
#             conf = float(box.conf[0])
            
#             # Draw rectangle
#             cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
            
#             # Add label
#             label = f"{class_name} {conf:.2f}"
#             cv2.putText(image, label, (x1, y1 - 10), 
#                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
#     # Add total counts to image
#     count_text = f"Dryer: {counts['Dryer']}, IRON: {counts['IRON']}"
#     cv2.putText(image, count_text, (10, 30), 
#                 cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    
#     # Save annotated image
#     output_path = f"detected_{os.path.basename(image_path)}"
#     cv2.imwrite(output_path, image)
    
#     return counts

# def main():
#     # Step 1: Train the model
#     print("Training model...")
#     model = train_model()
    
#     # Step 2: Initialize CSV
#     csv_path = initialize_csv()
#     print(f"CSV file initialized at {csv_path}")
    
#     # Step 3: Process test images
#     test_dir = "test/images"
#     if os.path.exists(test_dir):
#         for image_file in os.listdir(test_dir):
#             if image_file.lower().endswith(('.png', '.jpg', '.jpeg')):
#                 image_path = os.path.join(test_dir, image_file)
#                 print(f"\nProcessing {image_path}")
                
#                 # Detect and count objects
#                 counts = detect_and_count(model, image_path)
#                 if counts:
#                     # Update CSV with new counts
#                     update_csv_counts(counts)
#                     print(f"Updated counts: Dryer={counts['Dryer']}, IRON={counts['IRON']}")
    
#     print("\nProcessing complete!")

# if __name__ == "__main__":
#     main()